# Introduction

julie.tsitron@parks.nyc.gov 1/8/2020

This notebook is for testing various ArcGIS API for Python functions that allow for pushing/updating GIS data to production servers. Ultimately, the script from this notebook can be used as a template for the technical implementation (i.e., pushing clean data into production ESRI-based geodatabases) of Structures, Sites and Units, CPAs, and other Agency Data Model data products.

# Imports and Connections to DBs

In [1]:
import pandas as pd
import pyodbc
import sys
sys.path.append('../') ## <-- THIS IS THE PART THAT TELLS IT TO LOOK IN THE PARENT DIRECTORY
from IPM_Shared_Code.Python.geo_functions import read_geosql
from IPM_Shared_Code.Python.email_functions import get_contacts, read_template, send_email
from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor, SpatialDataFrame, FeatureLayer
# import requests
# from urllib3 import PoolManager
# http = PoolManager()
import json
# import time
from datetime import datetime
import arcgis
import utils
import urllib
import sqlalchemy
import os
import shapely
import smtplib
# from shapely.geometry import shape, mapping

In [2]:
config = utils.get_config('config.ini')

driver = config['srv']['driver']
vpipm_server = config['srv']['vpipm']
parksgis_dev_server = config['srv']['parksgis_dev']
parksgis_prod_server = config['srv']['parksgis_prod']
data_parks_server = config['srv']['data_parks']
structuresdb = config['db']['structuresdb']

portal_dev = config['url']['portal_dev']
portal_prod = config['url']['portal_prod']
structures_dev_url = config['url']['structures_dev']
structures_prod_url = config['url']['structures_prod']

In [3]:
params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' +
                                 vpipm_server + ';Database=' + structuresdb +
                                 ';Trusted_Connection=Yes;')
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [4]:
connection = engine.connect()

In [5]:
con = pyodbc.connect('Driver={' + driver + '};Server=' + data_parks_server +
                     ';Database=IPMDB;Trusted_Connection=Yes;')
con_vpipm = pyodbc.connect('Driver={' + driver + '};Server=' + vpipm_server +
                           ';Database=;Trusted_Connection=Yes;')

# Data

# Deal with Dates

## m/d/Y H:M:S format:

In [24]:
# structures_dev['COMMISSIONDATE'] = pd.to_datetime(
#     structures_dev['COMMISSIONDATE'],
#     errors='coerce').dt.strftime('%m/%d/%Y %H:%M:%S')

In [25]:
# structures_dev['COMMISSIONDATE']

In [26]:
# structures_dev['FEATURESTATUSCHANGEDATE'] = pd.to_datetime(
#     structures_dev['FEATURESTATUSCHANGEDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [27]:
# structures_dev['RETIREDDATE'] = pd.to_datetime(
#     structures_dev['RETIREDDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [28]:
# structures_dev.columns.values

## Delta Table from structuresdb (on vpipm)

In [29]:
# SPATIAL DATASET:
sql_str_deltas = 'select * FROM [structuresdb].[dbo].[tbl_delta_structures]'
struct_deltas = read_geosql(sql_str_deltas,
                            con_vpipm,
                            geom_raw='shape',
                            geom_col='geometry')

In [30]:
## Still need this ??

struct_deltas.rename(columns={
    'objectid': 'OBJECTID',
    'parks_id': 'SYSTEM',
    'bin': 'BIN',
    'bbl': 'BBL',
    'doitt_id': 'DOITT_ID',
    'ground_elevation': 'Ground_Elevation',
    'height_roof': 'Height_Roof',
    'alteration_year': 'Alteration_Year',
    'construction_year': 'Construction_Year',
    'demolition_year': 'Demolition_Year'
},
                     inplace=True)

In [31]:
struct_deltas.head()

,fid,OBJECTID,SYSTEM,BIN,BBL,DOITT_ID,Ground_Elevation,Height_Roof,Construction_Year,Alteration_Year,Demolition_Year,api_call,doitt_source,date_stamp,geometry
0,1,4596,Q376-BLG0193,4616184,4065070001,220471,70.0,13.890000,1952.0,NaN,None,U,current,2020-02-11 20:01:51.967,"POLYGON ((1033184.264980316 207904.7335185558,..."
1,2,4716,Q099-ZN18-BLG0361,4557975,4020180001,221004,27.0,12.789576,1939.0,NaN,None,U,current,2020-02-11 20:01:51.967,"POLYGON ((1026348.098534554 210512.7463471293,..."
2,3,4772,Q220C-03-BLG0454,4584331,4119820001,222114,33.0,15.530000,1931.0,NaN,None,U,current,2020-02-11 20:01:51.967,"POLYGON ((1039076.640006557 187146.0099442154,..."
3,4,5371,B385-BLG1612,3336973,3025560041,222674,12.0,14.370000,1936.0,NaN,None,U,current,2020-02-11 20:01:51.967,"POLYGON ((995133.3437069803 205243.7522919625,..."
4,5,5339,M107-BLG1568,1088091,1018190010,223660,22.0,13.222770,1961.0,NaN,None,U,current,2020-02-11 20:01:51.967,"POLYGON ((1002872.513596058 226117.4808856249,..."


In [32]:
# multipolygons

In [33]:
multipolygons = []
for i, row in struct_deltas.iterrows(): 
    if type(row['geometry'])==shapely.geometry.multipolygon.MultiPolygon:
#         print('hi')
        multipolygons.append(row['SYSTEM'])
#         print(i)
# print(row['SYSTEM'])
if len(multipolygons)!=0:
    ids = '\n'.join(multipolygons[:1])
#     print(type(ids))
    mssg = 'SYSTEM ID(s) of building(s) that are MultiPolygons:\n'+ids
    send_email('mycontacts.txt','multipoly_mssg.txt', subject = 'multipolygons', e=mssg)
struct_deltas = struct_deltas[~struct_deltas['SYSTEM'].isin(multipolygons)]

In [40]:
# def to_wkt(row):
#     return row.wkt

# Write Delta Table to geojson file

In [44]:
today = datetime.now().strftime('%Y%m%d')
snapshot = r'C:\\Projects\\AgencyDataModel\\Develop\\Structures\\delta_snapshots/' + today
if os.path.exists(snapshot):
    struct_deltas.to_file(snapshot + '/deltas.geojson',
                          encoding='utf-8',
                          driver='GeoJSON')
else:
    os.makedirs(snapshot)
    struct_deltas.to_file(snapshot + '/deltas.geojson',
                          encoding='utf-8',
                          driver='GeoJSON')

# Read geojson file to geojson object 

In [49]:
with open(snapshot+'/deltas.geojson') as data:
    geojson_deltas = json.load(data)

# Create arcgis featureSet from geojson object

In [50]:
fromJSON_deltas = arcgis.features.FeatureSet.from_geojson(geojson_deltas)

C:\ProgramData\Anaconda3\lib\site-packages\arcgis\features\feature.py:793: UserWarning: crs has been deprecated and will be ignored. Please see: https://tools.ietf.org/html/rfc7946#section-4 for more information.
  " see: https://tools.ietf.org/html/rfc7946#section-4 for" +\


# Connect to published dataset via GIS object

In [51]:
gis = GIS(url=portal_dev)

# Connect to feature layer directly

In [52]:
strct_lyr_url = structures_dev_url

In [53]:
lyr_structures = FeatureLayer(strct_lyr_url)
structures_features = lyr_structures.query()

In [54]:
structures_features

<FeatureSet> 2697 features

In [55]:
len(struct_deltas)

2075

# Make Edits

In [62]:
crsr = con_vpipm.cursor()
sql_stmt = '''
exec [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] 
'''
crsr.execute("{CALL [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] }")
crsr.commit()
crsr.close()

In [57]:
update_result = lyr_structures.edit_features(updates=fromJSON_deltas.features)

In [60]:
lyr_structures = FeatureLayer(strct_lyr_url)
structures_features = lyr_structures.query()